# **NUMERICAL METHODS FOR SIMULTANEOUS DIAGONALIZATION**

Algorithm from Bunse-Gerstner, A., Byers, R., & Mehrmann, V. (1993). Numerical Methods for Simultaneous Diagonalization. SIAM Journal on Matrix Analysis and Applications, 14(4), 927–949. doi:10.1137/0614062 

### **Libraries**

In [ ]:
import numpy as np 
from scipy.stats import ortho_group
from scipy.optimize import minimize

### **Matrix Functions**

In [ ]:

def normal_matrix(n):
    '''
    By generating a random orthonormal matrix and two random diagonal
    matrix creates two normal matrix that commute
    '''
    Q = ortho_group.rvs(dim = n)
    Q_transpose = np.transpose(Q)

    vector_A = np.random.choice([-1,1],n)
    vector_B = np.random.choice([-1,1],n)
   
    diag_A = np.diag(vector_A)
    diag_B = np.diag(vector_B)

    A = Q @ diag_A @ Q_transpose
    B = Q @ diag_B @ Q_transpose

    return A, B, Q

### **Algorithm Functions**

In [ ]:
def off_2(A, B):
    A_array = np.ravel(A)
    A_diag = np.diag(A)
    
    B_array = np.ravel(B)
    B_diag = np.diag(B)

    square_all = np.dot(A_array, A_array) + np.dot(B_array, B_array)
    square_diag = np.dot(A_diag, A_diag) + np.dot(B_diag, B_diag)

    off = square_all - square_diag

    return off

def create_M_ij(A,B,i,j):
    a_ii = A[i,i]
    a_c_ii = np.conjugate(A[i,i])
    a_ij = A[i,j]
    a_c_ij = np.conjugate(A[i,j])
    a_ji = A[j,i]
    a_c_ji = np.conjugate(A[j,i])
    a_jj = A[j,j]
    a_c_jj = np.conjugate(A[j,j])

    b_ii = B[i,i]
    b_c_ii = np.conjugate(B[i,i])
    b_ij = B[i,j]
    b_c_ij = np.conjugate(B[i,j])
    b_ji = B[j,i]
    b_c_ji = np.conjugate(B[j,i])
    b_jj = B[j,j]
    b_c_jj = np.conjugate(B[j,j])

    m_21 = (a_c_ii -a_c_jj)/np.sqrt(2)
    m_22 = (a_ii - a_jj)/np.sqrt(2)
    m_23 = (b_c_ii - b_c_jj)/np.sqrt(2)
    m_24 = (b_ii - b_jj)/np.sqrt(2)

    m_1 = np.array([a_c_ij, a_ji, b_c_ij, b_ji])
    m_2 = np.array([m_21, m_22, m_23, m_24])
    m_3 = np.array([(-1*a_c_ji),(-1*a_ij),(-1*b_c_ji),(-1*b_ij)])

    M = np.matrix([m_1, m_2, m_3])
    M = np.transpose(M)
    
    return M

### **Optimizing Functions**

In [ ]:
def objective(M, x):
    '''
    Objective Function
    '''
    theta = x[0]
    phi = x[1]

    c = np.cos(theta)
    s = np.exp(1j*phi)*np.sin(theta)

    z = np.array([[np.square(c)],[np.sqrt(2)*c*s],[np.square(s)]])

    return np.linalg.norm(M.dot(z))

def constraint(x):
    '''
    Constraint
    '''
    theta = x[0]
    phi = x[1]

    c = np.cos(theta)
    s = np.exp(1j*phi)*np.sin(theta)

    cons_function = np.square(abs(c)) + np.square(abs(s)) - 1
    constraint_1 = {'type':'eq', 'fun': cons_function}
    constraints = [constraint_1]

    return constraints

def bnds():
    '''
    Bounds
    '''
    b_1 = (-1*np.pi/4, np.pi/4)
    b_2 = (-1*np.pi, np.pi)

    bounds = (b_1, b_2)

    return bounds